In [11]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import datetime
import keras
print(keras.__version__)

Using TensorFlow backend.


2.0.5


In [1]:
import util, time

In [2]:
start = time.time()
df = util.create_sparse_matrix("newl.csv")
print("time", time.time() - start)
df.head()

time 235.56591391563416


user_action  device_android_browser  \
user_id hotel_id                                        
0       93335        0.999808                     0.0   
        99307        1.499712                     0.0   
1       93462        0.999688                     0.0   
        235238       0.999631                     0.0   
        4799063      0.999631                     0.0   

                  device_android_hybrid_app  device_android_native_app  \
user_id hotel_id                                                         
0       93335                           0.0                        0.0   
        99307                           0.0                        0.0   
1       93462                           0.0                        0.0   
        235238                          0.0                        0.0   
        4799063                         0.0                        0.0   

                  device_android_tablet_browser  \
user_id hotel_id                                  
0       93335                               0.0   
        99307                               0.0   
1       93462                               0.0   
        235238                              0.0   
        4799063                             0.0   

                  device_android_tablet_hybrid_app  \
user_id hotel_id                                     
0       93335                                  0.0   
        99307                                  0.0   
1       93462                                  0.0   
        235238                                 0.0   
        4799063                                0.0   

                  device_android_tablet_native_app  device_ipad_browser  \
user_id hotel_id                                                          
0       93335                                  0.0                  0.0   
        99307                                  0.0                  0.0   
1       93462                                  0.0                  0.0   
        235238                                 0.0                  0.0   
        4799063                                0.0                  0.0   

                  device_ipad_hybrid_app  device_ipad_native_app  ...  \
user_id hotel_id                                                  ...   
0       93335                        0.0                     0.0  ...   
        99307                        0.0                     0.0  ...   
1       93462                        0.0                     0.0  ...   
        235238                       0.0                     0.0  ...   
        4799063                      0.0                     0.0  ...   

                  parent_brand_name_Marriott International, Inc.  \
user_id hotel_id                                                   
0       93335                                                  0   
        99307                                                  0   
1       93462                                                  0   
        235238                                                 1   
        4799063                                                0   

                  parent_brand_name_Preferred Hotel Group  \
user_id hotel_id                                            
0       93335                                           0   
        99307                                           0   
1       93462                                           0   
        235238                                          0   
        4799063                                         0   

                  parent_brand_name_Wyndham Hotel Group  \
user_id hotel_id                                          
0       93335                                         0   
        99307                                         0   
1       93462                                         0   
        235238                                        0   
        4799063                                       0   

     

In [3]:
ACTIVITY_DATA = "/Users/neeraj/Documents/Hackathons/data_brown_datathon/TripAdvisor/activity_data.csv"
HOTEL_DATA = "/Users/neeraj/Documents/Hackathons/data_brown_datathon/TripAdvisor/hotel_data.csv"

In [7]:
## jayant's code
activity_df = pd.read_csv(ACTIVITY_DATA)
hotel_df = pd.read_csv(HOTEL_DATA).set_index('hotel_id')
activity_df.head()

to_timestamp = lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').timestamp()
activity_df['timestamp'] = activity_df['date'].apply(to_timestamp)
#PUT IN GEOLOCATOR LATER FOR HOTELS AND FOR USERS

def encode(a):
    encodings = {'view':1, 'price_click':2, 'hotel_website_click':3, 'booking':4}
    return encodings[a]

activity_df['user_action'] = activity_df['user_action'].apply(encode)
activity_df = activity_df.drop(columns = ['date','user_country','device'])

#VECTOR EMBEDDING OF HOTEL NAME USING NLP AS FEATURE LATER ON
categorical_features = ['city_name', 'hotel_type', 'brand_name', 'parent_brand_name']
nlp_features = ['hotel_name']
hotel_df = hotel_df['hotel_name']
#hotel_df = pd.get_dummies(hotel_df, columns = categorical_features)

df = activity_df.join(hotel_df, how = 'inner', on = 'hotel_id')
df.head()

,user_id,hotel_id,user_action,timestamp,hotel_name
0,314066,570101,1,1.546837e+09,La Quinta Inn Queens (New York City)
974,229655,570101,1,1.547442e+09,La Quinta Inn Queens (New York City)
2409,135830,570101,1,1.547960e+09,La Quinta Inn Queens (New York City)
2840,222524,570101,2,1.546924e+09,La Quinta Inn Queens (New York City)
4468,51835,570101,1,1.547096e+09,La Quinta Inn Queens (New York City)


In [8]:
len(df.user_id.unique()), len(df.hotel_id.unique())

(363619, 1057)

In [12]:
df.user_id = df.user_id.astype('category').cat.codes.values
df.hotel_id = df.hotel_id.astype('category').cat.codes.values

In [13]:
df.head()

,user_id,hotel_id,user_action,timestamp,hotel_name
0,314066,281,1,1.546837e+09,La Quinta Inn Queens (New York City)
974,229655,281,1,1.547442e+09,La Quinta Inn Queens (New York City)
2409,135830,281,1,1.547960e+09,La Quinta Inn Queens (New York City)
2840,222524,281,2,1.546924e+09,La Quinta Inn Queens (New York City)
4468,51835,281,1,1.547096e+09,La Quinta Inn Queens (New York City)


In [14]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)

In [ ]:
train.head()
test.head()

In [15]:
from IPython.display import SVG
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
n_users, n_movies = len(df.user_id.unique()), len(df.hotel_id.unique())
n_latent_factors = 3

In [17]:
from keras.models import Model
movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors, name='Movie-Embedding')(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)

user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors,name='User-Embedding')(user_input))

prod = keras.layers.merge([movie_vec, user_vec], mode='dot',name='DotProduct')
model = Model([user_input, movie_input], prod)
model.compile('adam', 'mean_squared_error')



/Users/neeraj/Environments/dash/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if __name__ == '__main__':
/Users/neeraj/Environments/dash/lib/python3.7/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [19]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Item (InputLayer)                (None, 1)             0                                            
____________________________________________________________________________________________________
User (InputLayer)                (None, 1)             0                                            
____________________________________________________________________________________________________
Movie-Embedding (Embedding)      (None, 1, 3)          3174        Item[0][0]                       
____________________________________________________________________________________________________
User-Embedding (Embedding)       (None, 1, 3)          1090860     User[0][0]                       
___________________________________________________________________________________________

In [21]:
history = model.fit([train.user_id, train.hotel_id], train.user_action, epochs=100, verbose=1)

Epoch 1/100
1261691/1261691 [==============================] - 652s - loss: 1.0849   
Epoch 2/100
1261691/1261691 [==============================] - 615s - loss: 0.4611   
Epoch 3/100
1261691/1261691 [==============================] - 609s - loss: 0.3401   
Epoch 4/100
 720096/1261691 [================>.............] - ETA: 269s - loss: 0.2881

KeyboardInterrupt: 

In [36]:
pd.Series(history.history['loss']).plot(logy=True)
plt.xlabel("Epoch")
plt.ylabel("Train Error")

NameError: name 'history' is not defined

In [23]:
y_hat = np.round(model.predict([test.user_id, test.hotel_id]),0)
y_true = test.user_action

from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_true, y_hat)

0.45611131718359155

In [ ]:
movie_embedding_learnt = model.get_layer(name='Movie-Embedding').get_weights()[0]
pd.DataFrame(movie_embedding_learnt).describe()

In [ ]:
user_embedding_learnt = model.get_layer(name='User-Embedding').get_weights()[0]
pd.DataFrame(user_embedding_learnt).describe()

In [ ]:
from keras.constraints import non_neg
movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors, name='NonNegMovie-Embedding', embeddings_constraint=non_neg())(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)

user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors,name='NonNegUser-Embedding',embeddings_constraint=non_neg())(user_input))

prod = keras.layers.merge([movie_vec, user_vec], mode='dot',name='DotProduct')
model = Model([user_input, movie_input], prod)
model.compile('adam', 'mean_squared_error')

## Neural Network

In [25]:
n_latent_factors_user = 5
n_latent_factors_movie = 8

movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding')(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)
movie_vec = keras.layers.Dropout(0.2)(movie_vec)


user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding')(user_input))
user_vec = keras.layers.Dropout(0.2)(user_vec)


concat = keras.layers.merge([movie_vec, user_vec], mode='concat',name='Concat')
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout')(dense)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(concat)
dropout_2 = keras.layers.Dropout(0.2,name='Dropout')(dense_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dense_2)
dropout_3 = keras.layers.Dropout(0.2,name='Dropout')(dense_3)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)


result = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)
adam = Adam(lr=0.005)
model = keras.models.Model([user_input, movie_input], result)
model.compile(optimizer=adam,loss= 'mean_absolute_error')

/Users/neeraj/Environments/dash/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app
/Users/neeraj/Environments/dash/lib/python3.7/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [26]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Item (InputLayer)                (None, 1)             0                                            
____________________________________________________________________________________________________
User (InputLayer)                (None, 1)             0                                            
____________________________________________________________________________________________________
Movie-Embedding (Embedding)      (None, 1, 8)          8464        Item[0][0]                       
____________________________________________________________________________________________________
User-Embedding (Embedding)       (None, 1, 5)          1818100     User[0][0]                       
___________________________________________________________________________________________

In [34]:
history = model.fit([train.user_id, train.hotel_id], train.user_action, epochs=5, verbose=1)

Epoch 1/5
1261691/1261691 [==============================] - 1440s - loss: 0.2772  
Epoch 2/5
1261691/1261691 [==============================] - 2556s - loss: 0.2767  
Epoch 3/5
 374816/1261691 [=======>......................] - ETA: 814s - loss: 0.2764

KeyboardInterrupt: 

In [35]:
y_hat_2 = np.round(model.predict([test.user_id, test.hotel_id]),0)
print(mean_absolute_error(y_true, y_hat_2))
print(mean_absolute_error(y_true, model.predict([test.user_id, test.hotel_id])))

0.2768409405782077
0.27699722395680876
